# モデル作成

In [1]:
#irisデータの読み込み
import sklearn
from sklearn.datasets import load_iris
iris = load_iris()
import pandas as pd
df = pd.DataFrame(iris.data, columns=["sepallength", "sepalwidth", "petallength", "petalwidth"])
df['target'] = iris.target
y = df[["target"]]
x = df[["sepallength", "sepalwidth", "petallength", "petalwidth"]]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

from sklearn.ensemble import RandomForestClassifier

# 学習する
model = RandomForestClassifier()
model.fit(x_train, y_train)

# スコアリングする
x_test_r=x_test.reset_index(drop=True)
x_test_r.join(pd.DataFrame(model.predict(x_test_r),columns=['prediction'])\
    .join(pd.DataFrame(model.predict_proba(x_test_r),columns=['Setosa_prob','Virginica_prob','Versicolor_prob']))
                ).head()

/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/ipykernel/__main__.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,sepallength,sepalwidth,petallength,petalwidth,prediction,Setosa_prob,Virginica_prob,Versicolor_prob
0,6.4,2.8,5.6,2.2,2,0.0,0.00,1.00
1,6.4,2.8,5.6,2.1,2,0.0,0.00,1.00
2,4.8,3.4,1.9,0.2,0,1.0,0.00,0.00
3,4.6,3.4,1.4,0.3,0,1.0,0.00,0.00
4,6.3,2.5,5.0,1.9,2,0.0,0.07,0.93


## ④WML Clientの接続

In [2]:
#デフォルトの1.0.95だとCP4Dの3.0.1に対応していない
#!pip install --upgrade  watson-machine-learning-client-V4==1.0.103
!pip install --upgrade  watson-machine-learning-client-V4

Requirement already up-to-date: watson-machine-learning-client-V4 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (1.0.103)


In [3]:
!pip show watson-machine-learning-client-V4

Name: watson-machine-learning-client-V4
Version: 1.0.103
Summary: Watson Machine Learning API Client
Home-page: http://wml-api-pyclient-v4.mybluemix.net
Author: IBM
Author-email: svagaral@in.ibm.com, nagiredd@in.ibm.com, amadeusz.masny1@ibm.com
License: BSD
Location: /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages
Requires: ibm-cos-sdk, tabulate, requests, certifi, pandas, urllib3, lomond
Required-by: 


Welcome to watson-machine-learning-client’s(V4) documentation! — watson-machine-learning-client(V4) 1.0 documentation
https://wml-api-pyclient-dev-v4.mybluemix.net/

In [4]:
import sys,os,os.path

token = os.environ['USER_ACCESS_TOKEN']
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
   "token": token,
   "instance_id" : "wml_local",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.0.1"
}

client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)



1.0.103


# ⑤プロジェクトにモデル登録

In [5]:

#プロジェクトに接続
project_id = os.environ['PROJECT_ID']
client.set.default_project(project_id)


'SUCCESS'

In [6]:
client.repository.ModelMetaNames.show()

------------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME            TYPE  REQUIRED  SCHEMA
NAME                      str   Y
DESCRIPTION               str   N
INPUT_DATA_SCHEMA         list  N         {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 'string', 'nullable(optional)': 'string'}]}
TRAINING_DATA_REFERENCES  list  N         [{'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'endpoint_url(required)': 'string', 'access_key_id(required)': 'string', 'secret_access_key(required)': 'string'},

In [7]:
#INPUT_DATA_SCHEMAを生成
import json
#x_fields=json.loads(x_train.to_json(orient='table'))['schema']['fields'][1:]
x_fields=[{'name': i, 'type': str(v)} for i,v in x_train.dtypes.items()]
x_fields

[{'name': 'sepallength', 'type': 'float64'},
 {'name': 'sepalwidth', 'type': 'float64'},
 {'name': 'petallength', 'type': 'float64'},
 {'name': 'petalwidth', 'type': 'float64'}]

In [8]:
#OUTPUT_DATA_SCHEMAを生成
#y_fields=[json.loads(y_train.to_json(orient='table'))['schema']['fields'][1:][0]]
#y_fields[0]['metadata']={'modeling_role': 'prediction'}
y_fields=[{'name': 'prediction', 'type': str(y_train.dtypes[0]),'metadata': {'modeling_role': 'prediction'}}]
y_fields.append({'name': 'probability', 'type': 'list','metadata': {'modeling_role': 'probability'}})
y_fields

[{'name': 'prediction',
  'type': 'int64',
  'metadata': {'modeling_role': 'prediction'}},
 {'name': 'probability',
  'type': 'list',
  'metadata': {'modeling_role': 'probability'}}]

In [9]:
print(sklearn.__version__) 

0.22.1


In [10]:
model_name = 'iris_skl_model'
#scikit learnが0.22の場合はRUNTIME_UIDではなくSOFTWARE_SPEC_UIDの指定が必要
sw_spec_uid = client.software_specifications.get_uid_by_name("scikit-learn_0.22-py3.6")

#モデルのメタデータを定義
pro_model_meta_props = {
    client.repository.ModelMetaNames.NAME: model_name,
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.22",
#    client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3"
    client.repository.ModelMetaNames.INPUT_DATA_SCHEMA:[{
        "id":"input1",
        "fields":x_fields
    }],
    client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: {
        "id":"output",
        "fields": y_fields}
}

#プロジェクトにモデルを保存
stored_pro_model_details = client.repository.store_model(model,
                                               meta_props=pro_model_meta_props,
                                               training_data=x_train,
                                               training_target=y_train)
#stored_model_details
pro_model_uid=stored_pro_model_details['metadata']['guid']
pro_model_uid

'a9c0e31b-fc8b-4ab5-bafe-45cf42c62e37'

In [11]:
client.repository.list_models()

------------------------------------  --------------  ------------------------  -----------------
GUID                                  NAME            CREATED                   TYPE
a9c0e31b-fc8b-4ab5-bafe-45cf42c62e37  iris_skl_model  2020-07-02T07:07:41.002Z  scikit-learn_0.22
------------------------------------  --------------  ------------------------  -----------------


# ⑥デプロイメントスペースにモデルを登録

In [12]:
client.spaces.list()

------------------------------------  ------------  ------------------------
GUID                                  NAME          CREATED
7402f0d1-b651-4302-9527-9e5ae7b040a2  test_dp       2020-07-01T09:34:55.140Z
d77a074e-7c37-4e5d-abd3-7d96684973a7  ds_hkwd_20Q2  2020-06-24T09:41:49.756Z
77057008-9e46-4fb2-bccb-11ebc7711416  deploy_hkwd   2020-06-23T09:46:26.908Z
------------------------------------  ------------  ------------------------


In [13]:
#関連付けられたデプロイメント・スペースのIDを取得
import json, requests
# get project info
r = requests.get(os.environ['RUNTIME_ENV_APSX_URL'] + "/v2/projects/" + os.environ['PROJECT_ID'], headers = {"Authorization": "Bearer " + os.environ['USER_ACCESS_TOKEN']})
wmlProjectCompute = [c for c in r.json()["entity"]["compute"] if c["type"] == "machine_learning"][0]
space_uid = wmlProjectCompute["properties"]["space_guid"]
print(space_uid)

7402f0d1-b651-4302-9527-9e5ae7b040a2


In [14]:
#デプロイメントスペースに接続
client.set.default_space(space_uid)

Unsetting the project_id ...


'SUCCESS'

In [15]:
# モデルのメタデータにデプロイメントスペースのIDを追加
ds_model_meta_props=pro_model_meta_props
ds_model_meta_props[client.repository.ModelMetaNames.SPACE_UID]= space_uid

#デプロイメントスペースにモデルを保存
stored_ds_model_details = client.repository.store_model(model,
                                               meta_props=ds_model_meta_props,
                                               training_data=x_train,
                                               training_target=y_train)
ds_model_uid = stored_ds_model_details["metadata"]["guid"]
ds_model_uid 

'a06d9646-8829-4a6f-9751-4b95779f2821'

In [16]:
client.repository.list_models()

------------------------------------  --------------  ------------------------  -----------------
GUID                                  NAME            CREATED                   TYPE
a06d9646-8829-4a6f-9751-4b95779f2821  iris_skl_model  2020-07-02T07:07:43.002Z  scikit-learn_0.22
------------------------------------  --------------  ------------------------  -----------------


# ⑦モデルをオンラインスコアリングとしてデプロイ

In [17]:
#client.repository.ModelMetaNames.show()
#client.deployments.metanames.DeploymentMetaNames()
#client.deployments.DeploymentMetaNames()
client.deployments.ConfigurationMetaNames.get()

['ASSET',
 'AUTO_REDEPLOY',
 'BATCH',
 'COMPUTE',
 'CUSTOM',
 'DESCRIPTION',
 'HARDWARE_SPEC',
 'HYBRID_PIPELINE_HARDWARE_SPECS',
 'NAME',
 'ONLINE',
 'R_SHINY',
 'SPACE_UID',
 'TAGS',
 'VIRTUAL']

In [18]:
deployment_on_name = 'iris_skl_model_rt'
# オンラインスコアリングのメタデータ定義
deploy_on_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: deployment_on_name,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# モデルのデプロイ
deployment_on_details = client.deployments.create(
    artifact_uid=ds_model_uid, 
    meta_props=deploy_on_meta_props)



#######################################################################################

Synchronous deployment creation for uid: 'a06d9646-8829-4a6f-9751-4b95779f2821' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='34230b0a-68c8-454a-a485-f18e9b98180b'
------------------------------------------------------------------------------------------------




In [19]:
deployment_on_uid = client.deployments.get_uid(deployment_on_details)
print('Deployment uid = {}'.format(deployment_on_uid))

Deployment uid = 34230b0a-68c8-454a-a485-f18e9b98180b


In [20]:
client.deployments.list()

------------------------------------  -----------------  -----  ------------------------  -------------
GUID                                  NAME               STATE  CREATED                   ARTIFACT_TYPE
34230b0a-68c8-454a-a485-f18e9b98180b  iris_skl_model_rt  ready  2020-07-02T07:07:44.341Z  model
------------------------------------  -----------------  -----  ------------------------  -------------


# スコアリング

In [21]:
from pprint import pprint
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA:[{
        "fields": x_test.columns.tolist(),
#        "values": [list(x_test.iloc[0]), list(x_test.iloc[1])]
        "values": x_test[0:2].values.tolist()
    }]
}
pprint(payload)

{'input_data': [{'fields': ['sepallength',
                            'sepalwidth',
                            'petallength',
                            'petalwidth'],
                 'values': [[6.4, 2.8, 5.6, 2.2], [6.4, 2.8, 5.6, 2.1]]}]}


In [22]:
from pprint import pprint
predict=client.deployments.score(deployment_on_uid, payload )
pprint(predict)

{'predictions': [{'fields': ['prediction', 'probability'],
                  'values': [[2, [0.0, 0.0, 1.0]], [2, [0.0, 0.0, 1.0]]]}]}


# クリーンアップ

In [23]:
#デプロイメントの削除
#client.deployments.delete(deployment_on_uid)
#デプロイメントスペースのモデル削除
#client.repository.delete(ds_model_uid)


In [24]:
#プロジェクトのモデル削除
#client.set.default_project(project_id)
#client.repository.delete(pro_model_uid)